# BC Assessment Propert Tax

### Download the BC Assessment Property Tax data

In [1]:
import sys
import pandas as pd
import requests
import zipfile
import os
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import math
from sklearn.preprocessing import MinMaxScaler
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

init_notebook_mode(connected=True)
bca_property_tax_dir = os.getcwd() + "/bca_property_tax/"

In [2]:
def download_and_extract_bc_assessment_csvs():
    link = 'http://dev.sfworx.com/van_tr.zip'
    zip_file_name = 'van_tr.zip'

    # Check if file has been already downloaded if not download it
    if not os.path.isfile(zip_file_name):
        print("Downloading %s" % zip_file_name)
        response = requests.get(link, stream=True)
        zip_file = open(zip_file_name, "wb")

        total_length = response.headers.get('content-length')
        dl = 0
        total_length = int(total_length)
        for chunk in response.iter_content(chunk_size=512):
            if chunk:  # filter out keep-alive new chunks
                dl += len(chunk)
                zip_file.write(chunk)
                done = int(50 * dl / total_length)
                sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50 - done)))
                sys.stdout.flush()
        zip_file.close()

    # Unzip the file
    zip_ref = zipfile.ZipFile(zip_file_name, 'r')
    zip_ref.extractall(bca_property_tax_dir)
    zip_ref.close()

### Process the downloaded files

In [3]:
# We have combined all the CSVs from BC assessment to get the total price in each area code.
def combine_csvs():
    bca_property_tax_combined_df = pd.DataFrame(columns=['AREA_CODE', 'HOUSE_PRICE', 'YEAR'])

    for file in os.listdir(bca_property_tax_dir):
        if file.endswith(".csv"):
            bc_assessment_df = pd.read_csv(bca_property_tax_dir + file)
            bc_assessment_df['HOUSE_PRICE'] =  bc_assessment_df.apply(lambda x: (x['CURRENT_LAND_VALUE']+x['CURRENT_IMPROVEMENT_VALUE']), axis = 1)
            bc_assessment_df = bc_assessment_df.dropna(subset=['PROPERTY_POSTAL_CODE', 'HOUSE_PRICE'])
            bc_assessment_df = bc_assessment_df[~bc_assessment_df['PROPERTY_POSTAL_CODE'].str.startswith('M')]
            bc_assessment_df[['AREA_CODE', 'LOCALITY_CODE']] = bc_assessment_df['PROPERTY_POSTAL_CODE'] \
                .str.split(expand=True)
            bc_assessment_df = bc_assessment_df.groupby('AREA_CODE')[['HOUSE_PRICE']].median().reset_index()
            bc_assessment_df['YEAR'] = file.title().split('.')[0]
            bca_property_tax_combined_df = pd.concat([bca_property_tax_combined_df, bc_assessment_df],
                                                     ignore_index=True)
    
    # Filter the DF with the area codes of Greater Vancouver area
    accepted_zip_codes = ['V3', 'V4', 'V5', 'V6', 'V7']
    bca_property_tax_combined_df = bca_property_tax_combined_df[bca_property_tax_combined_df['AREA_CODE'].str[:2].isin(accepted_zip_codes)]

    return bca_property_tax_combined_df

### HPI BenchMark Median Aggregation - By Year

In [4]:
def hpi_benchmark_index():
    hpi_benchmark_prices_df = pd.read_csv('HPI_benchmark_prices.csv')
    hpi_benchmark_prices_df = hpi_benchmark_prices_df[['Date', 'Composite_Benchmark']]
    hpi_benchmark_prices_df[['Month', 'YEAR']] = hpi_benchmark_prices_df['Date'].str.split(" ", expand=True)
    hpi_benchmark_prices_df = hpi_benchmark_prices_df.groupby('YEAR')[['Composite_Benchmark']].median().reset_index()
    hpi_benchmark_prices_df['YEAR'] = hpi_benchmark_prices_df['YEAR'].astype(int)
    return hpi_benchmark_prices_df

### Main function call

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

bca_property_prices_historical_df = pd.DataFrame()
final_csv_name = 'bca_property_prices_historical.csv'
if not os.path.isfile(final_csv_name):
    download_and_extract_bc_assessment_csvs()
    bca_property_prices_historical_df = combine_csvs()
    bca_property_prices_historical_df.to_csv('bca_property_prices_historical.csv')
else:
    bca_property_prices_historical_df = pd.read_csv(final_csv_name)

hpi_benchmark_median_df = hpi_benchmark_index()

# Plot the HPI Benchmark Price 2005-2019
trace = go.Scatter(
            x=hpi_benchmark_median_df['YEAR'],
            y=hpi_benchmark_median_df['Composite_Benchmark']
        )
hpi_graph_data = [trace]
        
layout = go.Layout(
    title='CREA HPI Benchmark Price 2005-2019',
    xaxis=dict(title='YEAR'),
    yaxis=dict(title='BENCHMARK PRICE')
)
fig = go.Figure(data=hpi_graph_data, layout=layout)
iplot(fig)

In [418]:
# accepted_zip_codes = ['V3', 'V4', 'V5', 'V6', 'V7']
# hello=pd.DataFrame()
# years=bca_property_prices_historical_df['YEAR'].unique().tolist()
# f=bca_property_prices_historical_df
# for i in accepted_zip_codes:
#     for j in years:
#         hello=hello.append(f[(f['AREA_CODE'].str[:2]== i) & (f['YEAR']==j)][:200])
        

1. /bca_property_tax/" will have the data with Updated HOUSE_PRICE (upon running the code --New update)
2. combine_csvs() Method is taking the csv from "/bca_property_tax/"
3. df23 is obtained from 'bca_property_prices_historical_df = combine_csvs()'

### We'll try to plot house price data points to see and identify the outliers

In [ ]:
def plot_data_points(data_frame, use_scaled_house_price = False):
    unique_years = data_frame['YEAR'].unique().tolist()
    unique_years.sort()
    
    for year in unique_years:   
        year_df = data_frame[data_frame['YEAR'] == year] 
        
        trace = go.Scatter(
            x=year_df['AREA_CODE'],
            y=year_df['HOUSE_PRICE'],
            name='Year {}'.format(year),
            mode='markers'
        )
        
        bca_property_data = [trace]
        layout = go.Layout(
            title='Area Wise House Price In Year {}'.format(year),
            xaxis=dict(title='AREA CODE'),
            yaxis=dict(title='HOUSE PRICE')
        )
        fig = go.Figure(data=bca_property_data, layout=layout)
        iplot(fig)

plot_data_points(bca_property_prices_historical_df)

### Detecting and removing outliers using IQR

In [ ]:
def detect_outliers_iqr(data):
    data = data.sort_values(ascending=True)
    q1, q3= np.percentile(data, [25,75])
    iqr = q3 - q1
    
    lower_bound = q1 -(1.5 * iqr) 
    upper_bound = q3 +(1.5 * iqr) 
    
    return lower_bound, upper_bound

lower_bound, upper_bound = detect_outliers_iqr(bca_property_prices_historical_df['HOUSE_PRICE'])
bca_property_prices_historical_without_outliers_df = bca_property_prices_historical_df[(bca_property_prices_historical_df['HOUSE_PRICE'] >= lower_bound) & (bca_property_prices_historical_df['HOUSE_PRICE'] <= upper_bound)]

### Scale the house price after outliers removal

In [421]:
bca_property_prices_with_hpi_benchmark_df = pd.merge(hpi_benchmark_median_df, bca_property_prices_historical_without_outliers_df, on='YEAR')

dffin = bca_property_prices_with_hpi_benchmark_df[['HOUSE_PRICE']]
# Scaling it based on Min and Max of Composite Benchmark
scaler = MinMaxScaler(feature_range=(int(min(bca_property_prices_with_hpi_benchmark_df['Composite_Benchmark'])),  int(max(bca_property_prices_with_hpi_benchmark_df['Composite_Benchmark']))))
bca_property_prices_with_hpi_benchmark_df[['scaled_HOUSE_PRICE']] = pd.DataFrame(scaler.fit_transform(dffin), columns=dffin.columns)

### Calculate the HPI Percent Difference

In [423]:
# Percentage Difference between Composite and Scaled Housing Price
bca_property_prices_with_hpi_benchmark_df['HPIpercent_diff'] = ((bca_property_prices_with_hpi_benchmark_df['HOUSE_PRICE'] - bca_property_prices_with_hpi_benchmark_df['Composite_Benchmark']) / bca_property_prices_with_hpi_benchmark_df['Composite_Benchmark']) * 100

greaterDF = pd.DataFrame(columns=['YEAR', 'Composite_Benchmark', 'AREA_CODE', 'HOUSE_PRICE'])
lesserDF = pd.DataFrame(columns=['YEAR', 'Composite_Benchmark', 'AREA_CODE', 'HOUSE_PRICE'])
count=0

# We're iterating over the unique areas and then checking and calculating total number of ovverences 
# where the HPIpercent_diff lies above the +10% or below the -10%. If a area is continously lying in this
# range more than 7 times, we are certain that the area is bubble proned.
for area in bca_property_prices_with_hpi_benchmark_df['AREA_CODE'].unique().tolist():
    x = bca_property_prices_with_hpi_benchmark_df[bca_property_prices_with_hpi_benchmark_df['AREA_CODE'] == area] 
    
    # Setting the Threshold percentage 10,-10 over here
    greater = x[x['HPIpercent_diff'] > 10]
    lesser = x[x['HPIpercent_diff'] < -10]
    total_count = greater['HPIpercent_diff'].count() + lesser['HPIpercent_diff'].count()
        
    # To check whether the current area is bubble prone or not. We've set a threshold frequency of 7.
    # Which means, if the area is continously appearing for more than 7 years in the range above 10 or below -10
    # HPI Percent difference, we're saying those areas are more bubble prone.    
    if total_count > 7 :
        greaterDF = greaterDF.append(greater)
        lesserDF = lesserDF.append(lesser)

# combine greaterDF , lesserDF
df24 = lesserDF.append(greaterDF)
df24.head()

,AREA_CODE,Composite_Benchmark,HOUSE_PRICE,HPIpercent_diff,Unnamed: 0,YEAR,scaled_HOUSE_PRICE
0,V5K,485850,420000.0,-13.553566,350.0,2006,527679.347826
3,V5N,485850,412750.0,-15.045796,353.0,2006,525936.458333
5,V5R,485850,405500.0,-16.538026,355.0,2006,524193.568841
6,V5S,485850,431100.0,-11.268910,356.0,2006,530347.771739
7,V5T,485850,246000.0,-49.367089,357.0,2006,485850.000000


### Getting the DF with Area, Frequency, Percentage diff median

In [424]:
#getting the frequency and Median of the data
df24['FREQUENCY'] = df24.groupby('AREA_CODE')['AREA_CODE'].transform('count')
df24['MEDIAN'] = df24.groupby('AREA_CODE')['HPIpercent_diff'].transform('median')
final_df = df24[['AREA_CODE', 'FREQUENCY', 'MEDIAN']].drop_duplicates()

# Sorting
final_df = final_df.sort_values(by=['MEDIAN'], ascending=False)
final_df.reset_index(drop=True)

,AREA_CODE,FREQUENCY,MEDIAN
0,V6N,11,167.423015
1,V6L,11,158.743922
2,V6M,11,149.918963
3,V6S,12,135.578606
4,V6R,12,134.913643
5,V6P,14,107.375545
6,V6C,13,83.912553
7,V5Y,11,53.597022
8,V5P,13,39.253767
9,V5W,12,34.999909


### Visualize the bubble prone areas

In [425]:
trace = go.Bar(x=final_df['AREA_CODE'], 
               y=final_df['FREQUENCY']
              )
freqeuncy_graph_data = [trace]
layout = go.Layout(
    title='Bubble prone areas in year 2006-2019',
    xaxis=dict(title='AREA CODE'),
    yaxis=dict(title='FREQUENCY')
)
fig = go.Figure(data=freqeuncy_graph_data, layout=layout)
iplot(fig)

trace = go.Scatter(x=final_df['AREA_CODE'], y=final_df['MEDIAN'])
median_graph_data = [trace]
layout = go.Layout(
    title='Median HPI percent difference in year 2006-2019',
    xaxis=dict(title='AREA CODE'),
    yaxis=dict(title='MEDIAN')
)
fig = go.Figure(data=median_graph_data, layout=layout)
iplot(fig)

### Visualise the area's HPI Percent Change Trends over years, which is maximum bubble prone

In [426]:
max_frequency = max(final_df['FREQUENCY'].values)
temp_df = final_df.loc[final_df['FREQUENCY'] == max_frequency].sort_values(by=['MEDIAN'], ascending=False)
area_code = temp_df.iloc[0][0]

bubbled_area_df = bca_property_prices_with_hpi_benchmark_df[bca_property_prices_with_hpi_benchmark_df['AREA_CODE'] == area_code]
graph_title = 'HPI Benchmark vs HPI Percent Difference 2005-2019 For {}'.format(area_code)

trace1 = go.Scatter(
    x=hpi_benchmark_median_df['YEAR'],
    y=hpi_benchmark_median_df['Composite_Benchmark'],
    name='HPI Benchmark Price {}'.format(area_code)
)
trace2 = go.Scatter(
    x=bubbled_area_df['YEAR'],
    y=bubbled_area_df['scaled_HOUSE_PRICE'],
    name='Scaled House Price {}'.format(area_code)
)

hpi_graph_data = [trace1, trace2]
layout = go.Layout(
    title=graph_title,
    xaxis=dict(title='YEAR'),
    yaxis=dict(title='HPI Benchmark vs Scaled House Price')
)
fig = go.Figure(data=hpi_graph_data, layout=layout)
iplot(fig)


# House Price Comparison on Top-5 Bubbled Areas - 3D Plot

In [38]:
#Loading the list of Bubble proned areas(Obtained from Analysis)
Bubble_Records = pd.read_csv('hpi_van_west.csv')
Bubble_Top5=Bubble_Records.sort_values(by=['MEDIAN'],ascending=False)[:5]
Bubble_list=Bubble_Top5['AREA_CODE'].tolist()
Bubble_Records.sort_values(by=['MEDIAN'],ascending=False)
fill_colors = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854']
Area_Filter = bca_property_prices_historical_df.groupby('AREA_CODE')
data = []
for Area, fill_color in zip(Bubble_list, fill_colors):
    group = Area_Filter.get_group(Area)
    years = group['YEAR'].tolist()
    length = len(years)
    Area_coords = [Area] * length
    median_hp = group['HOUSE_PRICE'].tolist()
    zeros = [0] * length
    
    data.append(dict(
        type='scatter3d',
        mode='lines',
        x=years + years[::-1] + [years[0]],
        y=Area_coords * 2 + [Area_coords[0]],
        z=median_hp + zeros + [median_hp[0]],
        name='',
        surfaceaxis=1,
        surfacecolor=fill_color,
        line=dict(
            color='black',
            width=4),))
layout = dict(
    title='Vancover-West Top-5 Bubble prone Areas',
    scene=dict(
        xaxis=dict(title='YEAR'),
        yaxis=dict(title='AREA'),
        zaxis=dict(title='House_Price'),
        camera=dict(
            eye=dict(x=-1.7, y=-1.7, z=0.5))))
fig = go.Figure(data=data, layout=layout)
iplot(fig,filename = "Vancover-West Top-5 Bubble prone Areas")